<a href="https://colab.research.google.com/github/bforoura/GENAI26/blob/main/Module2/chapter2_example2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Chat Interface with Long-Term Memroy**

In [1]:
# Install the needed libraries
!pip install -qU langchain-google-genai

# The SQL memory features
!pip install -U langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.3/53.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 713.3/713.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.1/489.1 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.9/234.9 kB 7.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.43.0, but you have google-auth 2.47.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 4.0 MB/s eta 0:00:00
  Attemptin

In [2]:
import os
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import SystemMessage
from langchain_community.chat_message_histories import SQLChatMessageHistory

# 1. Setup
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

llm = ChatGoogleGenerativeAI(
    model="gemini-3-flash-preview",
    temperature=1.0
)

# 2. Setup Persistent Long-Term Memory
# 'session_id' allows you to have multiple different chats in one database
# 'connection_string' creates a local file named 'joke_factory.db'
session_id = "science_comedian_v1"
chat_history = SQLChatMessageHistory(
    session_id=session_id,
    connection_string="sqlite:///joke_factory.db"
)

# 3. Handle the System Persona
# Since the DB might already have messages, we only add the SystemMessage
# if the history is empty.
if not chat_history.messages:
    chat_history.add_message(SystemMessage(content="You are a witty comedian who specializes in science jokes."))

print(f"--- Science Comedian Bot (Session: {session_id}) ---")
print(f"--- {len(chat_history.messages)} messages loaded from memory ---")

# 4. The Chat Loop
while True:
    user_input = input("You: ")

    if user_input.lower() in ["exit", "quit", "bye"]:
        print("Comedian: I'm 'splitting' like an atom! Goodbye!")
        break

    # Add user's message to the SQLite database
    chat_history.add_user_message(user_input)

    # Get the AI response
    # We pass the ENTIRE history from the database to Gemini
    response = llm.invoke(chat_history.messages)
    ai_text = response.content[0]['text']

    print(f"Comedian: {ai_text}")

    # Save the AI's response to the SQLite database
    chat_history.add_ai_message(ai_text)



/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py:3553: LangChainDeprecationWarning: `connection_string` was deprecated in LangChain 0.2.2 and will be removed in 1.0. Use connection instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


--- Science Comedian Bot (Session: science_comedian_v1) ---
--- 1 messages loaded from memory ---
You: That is not funny!
Comedian: Oh, I see. My humor is like a noble gas... it elicited **no reaction**.

Listen, I get it. Comedy is a lot like quantum physics: a joke can be both hilarious and terrible at the exact same time until someone actually observes it. Clearly, you’ve collapsed my wavefunction into a state of "meh."

But don’t blame me! Blame the second law of thermodynamics. I was trying to bring some order to the room, but entropy always wins in the end, and things just naturally trend toward disorder and bad puns.

Give me one more chance! I have a great joke about momentum, but it’s really hard to get started and even harder to stop. 

Or we could talk about Earth’s rotation? It really makes my day.
You: Let's spin that around for a while!
Comedian: Now we’re talking! I love your **angular momentum**. 

Just be careful—if we keep spinning this topic around, we might trigger 

# **Why this is Long-Term Memory**


* Every message is saved to **joke_factory.db** instantly.


* If you stop the code, wait for a while, and run it again, **chat_history.messages** will automatically contain everything you said earlier.


* You can download **joke_factory.db** to your computer and all your data is inside.


* If the comedian gets too annoying or the history gets too long, you can clear the database by running: *chat_history.clear()(